# Week 5: machine learning


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans #The k-means algorithm
from sklearn.model_selection import train_test_split #We need this to split the data


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

## Loading the CSV file
Started out with loading in the csv dataset and I dropped the empty cells within the dataframe.

In [2]:
df = pd.read_csv('voice.csv')
df = df.dropna()
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


## Counting the variables I want to predict
I am using the function .value_counts() for counting the total variables male and female. 

In [3]:
df['label'].value_counts()

female    1584
male      1584
Name: label, dtype: int64

## Finding correlations
Next I searched for the best correlation to decide which 7 variables I am going to use for the prediction model. In order to find the best correlation with the variable 'label' I turned the labels male/female into 0's and 1's. 

In [4]:
# sns.pairplot(df)
# plt.show() #using Matplotlib to show the plot

In [5]:
label_encoder = LabelEncoder()

df[['label']] = label_encoder.fit_transform(df['label'])
dict(enumerate(label_encoder.classes_))

{0: 'female', 1: 'male'}

In [6]:
df_find_correlations = df.corr()
df_find_correlations["label"].sort_values(ascending=True)

meanfun    -0.833921
Q25        -0.511455
meanfreq   -0.337415
centroid   -0.337415
median     -0.283919
maxdom     -0.195657
mindom     -0.194974
dfrange    -0.192213
meandom    -0.191067
mode       -0.171775
maxfun     -0.166461
minfun     -0.136692
modindx     0.030801
skew        0.036627
Q75         0.066906
kurt        0.087195
sfm         0.357499
sd          0.479539
sp.ent      0.490552
IQR         0.618916
label       1.000000
Name: label, dtype: float64

Here you can see that the variables: meanfun, Q25 have the strongest negatative correlation and were meanfun has a strong correlation and Q25 a moderate correlation with the variable 'label'. 

Futher the variables: IQR, sp.ent have the strongest positive correlation. Here IQR has a strong correlation and sp.ent a moderate. 

In order to get a valid prediction I didn't want to work with only 4 variables. That is why I chose to include the variables: sd, sfm and meanfreq when using Knn to create a prediction model. These were the 'next in line' when it comes to a strong correlation. 



Next I created a subset with the chosen variables 

In [7]:
df_subset = df[["meanfun", "Q25", "IQR" , "sp.ent", "sd", "sfm", "meanfreq", "label"]]
df_subset.head()

,meanfun,Q25,IQR,sp.ent,sd,sfm,meanfreq,label
0,0.084279,0.015071,0.075122,0.893369,0.064241,0.491918,0.059781,1
1,0.107937,0.019414,0.073252,0.892193,0.067310,0.513724,0.066009,1
2,0.098706,0.008701,0.123207,0.846389,0.083829,0.478905,0.077316,1
3,0.088965,0.096582,0.111374,0.963322,0.072111,0.727232,0.151228,1
4,0.106398,0.078720,0.127325,0.971955,0.079146,0.783568,0.135120,1


## Working with k-nearest neigbor

Next I created the X and Y matrix and normalized the X matrix to put everything on the same scale. This normalization of the variables is needed when using the k-NN algorithm.

The k-NN algorithm is an algorithm that can be used for making a prodiction model for both classification and regression. In this case we have a classification 'problem', male or female.


In [8]:
from sklearn.preprocessing import normalize 

X = df[["meanfun", "Q25", "IQR" , "sp.ent", "sd", "sfm", "meanfreq"]] 
X = normalize(X) 
y = df['label'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [9]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=5)
knn = knn.fit(X_train, y_train) 
knn.score(X_test, y_test) 

0.9758149316508938

98% of the label male or female is predicted accuratly. However I am wondering if I did something wrong here, because this is an extremely high accuracy. 

## Confusion matrix

I order for me to find out how well the model predicts the different label a made a confusion matrix. 

In [10]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) 
cm = confusion_matrix(y_test, y_test_pred) 
cm

array([[448,   9],
       [ 14, 480]])

In [11]:
conf_matrix = pd.DataFrame(cm, index=['male', 'female'], columns = ['male_p', 'female_p']) 
conf_matrix

,male_p,female_p
male,448,9
female,14,480


Here you can see that from the (448+9) 457 males 9 were predicted wrong. From the (14+480) 494 females 14 were predicted wrong.

The _recall_ and _precision_ for the matrix is:

$recall = \frac{448}{448 + 9} = .98$

$precision = \frac{480}{480 + 14} = .97$

